# Testbench File

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs

In [6]:
from qiskit_metal.qlibrary.qubits.transmon_pocket_cl import TransmonPocketCL
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

from qiskit_metal.qlibrary.interconnects.meandered import RouteMeander
from qiskit_metal.qlibrary.interconnects.pathfinder import RoutePathfinder

from qiskit_metal.qlibrary.passives.cap_three_fingers import CapThreeFingers
from qiskit_metal.qlibrary.passives.launchpad_wb import LaunchpadWirebond

In [3]:
design = metal.designs.DesignPlanar()

gui = metal.MetalGUI(design)

In [4]:
design.chips.main.size['size_x'] = '9mm'
design.chips.main.size['size_y'] = '3mm'
# wide cpw 
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um' 

design.overwrite_enabled = True
design.chips.main

{'material': 'silicon',
 'layer_start': '0',
 'layer_end': '2048',
 'size': {'center_x': '0.0mm',
  'center_y': '0.0mm',
  'center_z': '0.0mm',
  'size_x': '9mm',
  'size_y': '3mm',
  'size_z': '-750um',
  'sample_holder_top': '890um',
  'sample_holder_bottom': '1650um'}}

In [7]:
## Basic operation
options = dict(
    pad_width = '450 um',
    pad_height= '100 um',
    pad_gap = '30um',
    
    pocket_height = '650 um',
    cl_pocket_edge = '180',
    connection_pads = dict(
        R = dict(loc_W=+1, loc_H=+1, pocket_rise = '50um', pad_cpw_shift = '8um', pad_gap = '30 um', pad_width = '93um'),
        L = dict(loc_W=-1, loc_H=-1, pocket_rise = '50um', pad_cpw_shift = '8um', pad_gap = '30 um', pad_width = '93um'),
        B = dict(loc_W=+1, loc_H=-1, pocket_rise = '50um', pad_cpw_shift = '8um', pad_gap = '30 um', pad_width = '93um')
  
    ))

Q1 = TransmonPocket(design,'Q1', options = dict(
        pos_x='-1.7mm',
        pos_y='+0mm',
        gds_cell_name = 'TOP',
        hfss_inductance = '11.8nH',
        q3d_inductance =  '11.8nH',
        **options))

Q2 = TransmonPocket(design,'Q2', options = dict(
        pos_x='+1.7mm',
        pos_y='+0mm',
        gds_cell_name = 'TOP',
        hfss_inductance = '10.9nH',
        q3d_inductance =  '10.9nH',
        orientation = '180',
        **options))

Q3 = TransmonPocket(design,'Q3', options = dict(
        pos_x='+0.0mm',
        pos_y='+1.5mm',
        gds_cell_name = 'TOP',
        hfss_inductance = '12.8nH',
        q3d_inductance =  '12.8nH',
        orientation = '270',
        **options))


gui.rebuild()
gui.autoscale()

In [8]:
# Connect two pins with a CPW
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

RouteMeander.get_template_options(design)


options = Dict(
        fillet = '80um',
        lead=Dict(
            start_straight='100um',
            end_straight='100um'),
        trace_gap='9um',
        trace_width='15um')


cpw1 = connect('cpw1', 'Q1', 'B', 'Q2', 'R', '8.4 mm', f'-200um')
cpw2 = connect('cpw2', 'Q2', 'B', 'Q3', 'R', '8.2 mm', f'-900um')
cpw3 = connect('cpw3', 'Q3', 'B', 'Q1', 'R', '8.0 mm', f'-300um')

gui.rebuild()
gui.autoscale()

In [9]:
# Capacitor for readout resonators (3-finger type)
cap_Q1 = CapThreeFingers(design, 'Cap_Q1', options = dict(pos_x = '-3.2 mm',
                                                          pos_y = '0.5mm',
                                                         orientation = '90',
                                                         finger_length = '40 um',
                                                         trace_width='15 um'))
cap_Q2 = CapThreeFingers(design, 'Cap_Q2', options = dict(pos_x = '+3.2 mm',
                                                          pos_y = '-0.5 mm',
                                                         orientation = '-90',
                                                         finger_length = '40 um',
                                                         trace_width='15 um'))
cap_Q3 = CapThreeFingers(design, 'Cap_Q3', options = dict(pos_x = '0.5 mm',
                                                          pos_y = '3.2mm',
                                                         orientation = '-90',
                                                         finger_length = '40 um',
                                                         trace_width='15 um'))


gui.rebuild()
gui.autoscale()

In [10]:
# Readout resonators (for Q1 and Q2)
RO_Q1 = RouteMeander(design, 'RO_Q1', 
                    options = dict(pin_inputs = Dict(
                            start_pin = Dict(component = 'Q1',pin = 'L'),
                              end_pin = Dict(component = 'Cap_Q1',pin = 'b')),
                    lead = Dict(start_straight = '100 um',
                                end_straight = '100 um'),
                    meander = Dict(asymmetry = '-50 um'),
                    fillet = '90 um',
                    total_length = '7.5 mm'))

RO_Q2 = RouteMeander(design, 'RO_Q2', 
                    options = dict(pin_inputs = Dict(
                            start_pin = Dict(component = 'Q2',pin = 'L'),
                              end_pin = Dict(component = 'Cap_Q2',pin = 'b')),
                    lead = Dict(start_straight = '100 um',
                                end_straight = '100 um'),
                    meander = Dict(asymmetry = '-50 um'),
                    fillet = '90 um',
                    total_length = '7.3 mm'))

RO_Q3 = RouteMeander(design, 'RO_Q3', 
                    options = dict(pin_inputs = Dict(
                            start_pin = Dict(component = 'Q3',pin = 'L'),
                              end_pin = Dict(component = 'Cap_Q3',pin = 'b')),
                    lead = Dict(start_straight = '400 um',
                                end_straight = '100 um'),
                    meander = Dict(asymmetry = '-50 um'),
                    fillet = '90 um',
                    total_length = '7.1 mm'))

gui.rebuild()
gui.autoscale()

In [44]:
## 여기까지 geometry

## 2. Render into Ansys Q3D

In [11]:
fourq_q3d = design.renderers.q3d

Skip the next cell if Ansys is already open and with a project loaded

In [31]:
#fourq_q3d.open_ansys()
fourq_q3d.open_ansys(path='C:\Program Files\AnsysEM\AnsysEM21.1\Win64')

**Wait for Ansys to open up completely before proceeding to the next cell**

In [19]:
fourq_q3d.connect_ansys()
#fourq_q3d.add_q3d_design("Q3dMetalDesign")

INFO 12:19PM [connect_project]: Connecting to Ansys Desktop API...
INFO 12:19PM [load_ansys_project]: 	Opened Ansys App
INFO 12:19PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 12:19PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/LeeJaeho/Documents/Ansoft/
	Project:   Project1
INFO 12:19PM [connect_design]: 	Opened active design
	Design:    Q3DDesign2 [Solution type: Q3D]
WARNING 12:19PM [connect_setup]: 	No design setup detected.
INFO 12:19PM [connect]: 	Connection to Ansys established successfully. 😀 



Render a single qubit with 2 endcaps

In [20]:
#fourq_q3d.render_design([], [])
#fourq_q3d.render_design(['Q2'], [('Q2', 'R'), ('Q2', 'L')])
#fourq_q3d.render_design(['Q1', 'Cap_Q1','RO_Q1'], [('Q1', 'R'), ('Q1', 'L'), ('Cap_Q1', 'a'), ('Cap_Q1', 'b')])
#fourq_q3d.render_design(['Cap_Q1'], [('Cap_Q1', 'a'), ('Cap_Q1', 'b')])
#fourq_q3d.render_design(['Q3'], [('Q3', 'R'), ('Q3', 'L'), ('Q3', 'B')])
#fourq_q3d.render_design(['Q2'], [('Q2', 'R'), ('Q2', 'L'), ('Q2', 'B')])
fourq_q3d.render_design(['Q1'], [('Q1', 'R'), ('Q1', 'L'), ('Q1', 'B')])

## 3. Create a new solution setup in Ansys Q3D and obtain the capacitance matrix

In [21]:
#     Add a solution setup:
fourq_q3d.add_q3d_setup()
#     Example of how to change defaults (just an example. We will not use this setup):
#fourq_q3d.add_q3d_setup(name = 'Setup', max_passes = 20)

Optional keyword parameters and their defaults, listed below, represent parameters in this Q3D solution setup. Running fourq_q3d.add_q3d_setup() is equivalent to going to the Project Manager panel in Ansys, right clicking on Analysis within the active Q3D design, selecting "Add Solution Setup...", and choosing/entering default values in the resulting popup window. To easily keep track of different solution setups, give each of them a different name, otherwise, Ansys will simply append numbers to the end of duplicate names. <br><br>
freq_ghz: float = 5. <br>
name: str = "Setup" <br>
save_fields: bool = False <br>
enabled: bool = True <br>
max_passes: int = 15 <br>
min_passes: int = 2 <br>
min_converged_passes: int = 2 <br>
percent_error: float = 0.5 <br>
percent_refinement: int = 30 <br>
auto_increase_solution_order: bool = True <br>
solution_order: str = 'High' <br>
solver_type: str = 'Iterative <br><br>
These defaults can be modified by including the keywords and their new values inside the parentheses. For example, to increase the maximum number of passes from the default value of 15 up to 20 while leaving all else unchanged, replace the cell above with the following: <br><br>
fourq_q3d.add_q3d_setup(max_passes = 20)

In [22]:
# Analyze said solution setup.
fourq_q3d.analyze_setup("Setup")

INFO 12:20PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 12:20PM [analyze]: Analyzing setup Setup


In [23]:
# Using the analysis results, get its capacitance matrix as a dataframe.
fourq_q3d.get_capacitance_matrix()

B_connector_pad_Q1  ground_main_plane  L_connector_pad_Q1  \
B_connector_pad_Q1            41.27340          -30.86848            -0.20922   
ground_main_plane            -30.86848          237.94581           -30.95335   
L_connector_pad_Q1            -0.20922          -30.95335            41.27875   
pad_bot_Q1                    -8.01576          -39.83042            -8.03007   
pad_top_Q1                    -1.22248          -43.42213            -1.25033   
R_connector_pad_Q1            -0.21366          -31.00903            -0.08717   

                    pad_bot_Q1  pad_top_Q1  R_connector_pad_Q1  
B_connector_pad_Q1    -8.01576    -1.22248            -0.21366  
ground_main_plane    -39.83042   -43.42213           -31.00903  
L_connector_pad_Q1    -8.03007    -1.25033            -0.08717  
pad_bot_Q1            90.64188   -30.81362            -1.17315  
pad_top_Q1           -30.81362    87.66029            -8.02739  
R_connector_pad_Q1    -1.17315    -8.02739            41.25882

The optional keyword parameters of get_capacitance_matrix are given below: <br><br>
variation: str = '' <br>
solution_kind: str = 'AdaptivePass' <br>
pass_number: int = 3

In [24]:
# Run lumped oscillator model (LOM) simulations and save results in dict_lom.b
dict_lom = fourq_q3d.lumped_oscillator_vs_passes(11.8, 0, 3, 7.5, [6.5, 6.5], 8)
#dict_lom = fourq_q3d.lumped_oscillator_vs_passes(10.9, 0, 3, 7.7, [6.5, 6.5], 8)
#dict_lom = fourq_q3d.lumped_oscillator_vs_passes(12.8, 0, 3, 7.3, [6.5, 6.5], 8)
dict_lom

Pass number:  1
[3, 4] [5 0 1]
Predicted Values

Transmon Properties
f_Q 5.872726 [GHz]
EC 352.332086 [MHz]
EJ 13.847085 [GHz]
alpha -414.259636 [MHz]
dispersion 96.741064 [KHz]
Lq 0.000055 [nH]
Cq 54.977190 [fF]
T1 108.824528 [us]

**Coupling Properties**

tCqbus1 -3.513534 [fF]
gbus1_in_MHz -61.996381 [MHz]
χ_bus1 -0.976934 [MHz]
1/T1bus1 839.793211 [Hz]
T1bus1 189.516825 [us]

tCqbus2 3.296518 [fF]
gbus2_in_MHz 50.494547 [MHz]
χ_bus2 -3.247692 [MHz]
1/T1bus2 379.574413 [Hz]
T1bus2 419.298398 [us]

tCqbus3 -2.670324 [fF]
gbus3_in_MHz -40.411948 [MHz]
χ_bus3 -2.080202 [MHz]
1/T1bus3 243.123838 [Hz]
T1bus3 654.625001 [us]
Bus-Bus Couplings
gbus1_2 2.137252 [MHz]
gbus1_3 93.764408 [MHz]
gbus2_3 82.497024 [MHz]
Pass number:  2
[3, 4] [5 0 1]
Predicted Values

Transmon Properties
f_Q 5.815274 [GHz]
EC 344.973663 [MHz]
EJ 13.847085 [GHz]
alpha -404.693512 [MHz]
dispersion 80.761058 [KHz]
Lq 0.000056 [nH]
Cq 56.149875 [fF]
T1 126.534939 [us]

**Coupling Properties**

tCqbus1 -3.538002 [fF]


fQ          EC         EJ       alpha dispersion  \
1  5.872726  352.332086  13.847085 -414.259636  96.741064   
2  5.815274  344.973663  13.847085 -404.693512  80.761058   
3  5.703541  330.919722  13.847085 -386.531957   56.24726   
4  5.608632  319.245944  13.847085  -371.55386  40.892556   
5   5.55636  312.919289  13.847085 -363.476874  34.145254   
6  5.531131  309.891796  13.847085 -359.621804  31.260358   
7  5.506596  306.963643  13.847085 -355.899374  28.666037   

                                                gbus  \
1  [-61.996381117660825, 50.49454725226901, -40.4...   
2  [-61.407830185883505, 47.81048548927905, -41.6...   
3  [-59.80668459690721, 47.693441193262174, -42.7...   
4  [-60.584840706730176, 47.43123302968917, -41.7...   
5  [-59.656066538665655, 47.1813677683138, -41.89...   
6  [-60.06966618495659, 47.139881022115226, -41.4...   
7  [-60.5299780425607, 47.82912705791243, -42.389...   

                                          chi_in_MHz    χr MHz     gr MHz  
1  [-0.9769340944282228, -3.247691896489857, -2.0...  0.976934  61.996381  
2  [-0.8848139074236877, -2.4928359524815833, -1....  0.884814  61.407830  
3  [-0.7214314576714657, -1.878522926667776, -1.5...  0.721431  59.806685  
4  [-0.6536218742577712, -1.4968337330995247, -1....  0.653622  60.584841  
5  [-0.5925509588585363, -1.3234584642452927, -1....  0.592551  59.656067  
6  [-0.5818200197546097, -1.2531193498458448, -0....  0.581820  60.069666  
7  [-0.572732551552942, -1.2264458347226463, -0.9...  0.572733  60.529978

In [25]:
fourq_q3d.plot_convergence_main(dict_lom)
fourq_q3d.plot_convergence_chi(dict_lom)

INFO 12:21PM [hfss_report_full_convergence]: Creating report for variation 0


Design "Q3DDesign2" info:
	# eigenmodes    0
	# variations    1


INFO 12:21PM [__del__]: Disconnected from Ansys HFSS


<Figure size 990x385 with 2 Axes>

In [24]:
fourq_q3d.disconnect_ansys()